In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

def ycb_resource(name):
    global ycb
    return FindResourceOrThrow(ycb[name])

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

In [5]:
class Stage(Enum):
    INIT = 0
    TO_PREPICK = 1
    TO_PICK = 2
    PICKING = 3
    TO_POSTPICK = 4
    TO_INIT = 5

        
class FindGrasp(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        
        self.nq = 7
        self.panda_grasp_position_output_port = self.DeclareVectorOutputPort(
                                                                    "panda_grasp_position", 
                                                                    BasicVector(self.nq),
                                                                    self.DoCalcOutput)
        self.station = station
        self.station_context = station_context
        self.rng = np.random.default_rng()
        self.q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
        self.max_samples = 100
          
    def DoCalcOutput(self, context, output):
        start_time = time.time()
        ik_station, body_infos = create_welded_station(self.station, station_context)
        ik_station_context = ik_station.CreateDefaultContext()
        q, cost = grasp_pose(body_infos[0], ik_station, ik_station_context)
        print(f"q: {q}\ncost {cost}")
        output.set_value(q)
        

class Picker(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        
        self.nq = 7
        
        self.grasp_position_input_port = self.DeclareVectorInputPort("grasp_position", BasicVector(self.nq))
        self.panda_position_input_port = self.DeclareVectorInputPort("panda_position", 
                BasicVector(self.nq))
        self.panda_position_command_output_port = self.DeclareVectorOutputPort(
                "panda_position_command", BasicVector(self.nq),
                self.DoCalcOutput)
        self.hand_command_output_port = self.DeclareVectorOutputPort("hand_position_command", BasicVector(1),
                                                                    self.CalcHandOutput)
        
        self.station = station
        self.station_context = station_context
        self.plant = station.get_multibody_plant()
        self.panda = self.plant.GetModelInstanceByName("panda")
        self.hand = self.plant.GetModelInstanceByName("hand")
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = station.get_scene_graph()
        self.scene_graph_context = station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        
        self.q_S = None
        self.q_G = None
        self.q_P = None
        
        self.panda_traj = None
        self.hand_traj = None
        
        self.X_Pregrasp = None
        self.X_Grasp = None
        
        self.stage = Stage.INIT # Stage(0)
        self.setup_time = 0.5
        
        self.q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
        
        self.p_tol = 0.01*np.ones(3)
        self.theta_tol = 0.05
        
        self.cloud = None
        
        
    def CalcHandOutput(self, context, output):
        time = context.get_time()
        if (self.hand_traj is None) or (time > self.hand_traj.end_time()):
            output.set_value([0])
        else:
            output.set_value(self.hand_traj.value(time).flatten())
        
    def DoCalcOutput(self, context, output):
        time = context.get_time()
        if time < self.setup_time: # allow time for objects to settle
            self.q_S = self.EvalVectorInput(context, self.panda_position_input_port.get_index()).get_value()
            output.set_value(self.q_S)
            return
        
        if self.stage == Stage.INIT:
            print("INITIALIZATION")
            self.q_G = self.EvalVectorInput(context, self.grasp_position_input_port.get_index()).get_value()
            #self.q_P, self.X_Pregrasp, self.X_Grasp = self.pregrasp_q()
            #print(self.X_Pregrasp, self.X_Grasp)

            self.panda_traj = self.rrt_trajectory(self.q_S, time, self.q_G, time + 3)
            self.hand_traj = self.first_order_traj([0.0], time, [0.08], time + 3)
            self.stage = Stage.TO_PICK
            print("TO PREPICK")
            
        if self.stage == Stage.TO_PICK:
            
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.PICKING
                print("TO PICK")
                #self.panda_traj = self.rrt_trajectory(self.q_P, time, self.q_G, time + 2)
                #self.panda_traj = self.straight_line_traj(self.q_P, self.X_Pregrasp, time, 
                                                          #self.X_Grasp, time + 2)
                #self.hand_traj = self.first_order_traj([0.08], time, [0.08], time +2)
                
                self.hand_traj = self.first_order_traj([0.08], time, [0.0], time +2)
                self.panda_traj = self.first_order_traj(self.q_G, time, self.q_G, time +2)
        
        """
        if self.stage == Stage.TO_PICK:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.PICKING
                print("PICKING")
                self.hand_traj = self.first_order_traj([0.08], time, [0.0], time +2)
                self.panda_traj = self.first_order_traj(self.q_G, time, self.q_G, time +2)
        """
        
        if self.stage == Stage.PICKING:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_POSTPICK
                print("TO POSTPICK")
                self.panda_traj = self.rrt_trajectory(self.q_G, time, self.q_S, time + 3)
                self.hand_traj = self.first_order_traj([0.0], time, [0.0], time + 3)
                
        if self.stage == Stage.TO_POSTPICK:
            if time <= self.panda_traj.end_time():
                output.set_value(self.panda_traj.value(time).flatten())
            else:
                pass
                
            
    def straight_line_traj(self, q_start, X_start, start_time, X_end, end_time, N = 10):
        # interpolate between X_start and X_end (we want to maintain same gripper oritentation)
        times = np.array([start_time, end_time])
        ps = np.array([X_start.translation(), X_end.translation()])
        p_traj = PiecewisePolynomial.FirstOrderHold(times, ps.T)
        # solve ik problem at each time 
        times = np.linspace(start_time, end_time, N)
        qs = []
        R_WG = X_end.rotation()
        for i,t in enumerate(times):
            p_WG = p_traj.value(t)
            X_WG = RigidTransform(R_WG, p_WG)
            if i == 0:
                warm_start = q_start
            else:
                warm_start = qs[-1]
                
            q = self.X_WG_to_q(X_WG, warm_start = warm_start)
            qs.append(q)
            
        qs = np.array(qs)
        return PiecewisePolynomial.CubicShapePreserving(times, qs.T)
                
    def first_order_traj(self, q_start, start_time, q_end, end_time):
        time = np.array([start_time, end_time])
        qs = np.array([q_start, q_end])
        return PiecewisePolynomial.FirstOrderHold(time, qs.T)
            
    def rrt_trajectory(self, q_start, start_time, q_goal, goal_time):
        
        ik_station, _ = create_welded_station(self.station, self.station_context)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        panda = plant.GetModelInstanceByName("panda")
        hand = plant.GetModelInstanceByName("hand")
        scene_graph = ik_station.get_scene_graph()
        scene_graph_context = ik_station.GetSubsystemContext(scene_graph, ik_station_context)
        query_output_port = scene_graph.GetOutputPort("query")
        
        
        print("panda ids:")
        ids = plant.GetBodyIndices(panda)
        for i in ids:
            for col_id in plant.GetCollisionGeometriesForBody(plant.get_body(i)):
                print(col_id)
        print("hand ids")
        ids = plant.GetBodyIndices(hand)
        for i in ids:
            for col_id in plant.GetCollisionGeometriesForBody(plant.get_body(i)):
                print(col_id)
        print("table ids")
        ids = plant.GetBodyIndices(plant.GetModelInstanceByName("table"))
        for i in ids:
            for col_id in plant.GetCollisionGeometriesForBody(plant.get_body(i)):
                print(col_id)
        print("object ids")
        ids = plant.GetBodyIndices(plant.GetModelInstanceByName("soup"))
        for i in ids:
            body = plant.get_body(i)
            print(body.EvalPoseInWorld(plant_context))
            for col_id in plant.GetCollisionGeometriesForBody(plant.get_body(i)):
                print(col_id)
        print()
        
        def is_colliding(q):
            """ returns True if the configuration q results in a collision,
            False otherwise
            """
            # forwards kinematics (setting the position of the panda)
            plant.SetPositions(plant_context, panda, q)
            query_object = query_output_port.Eval(scene_graph_context)
            pairs = query_object.ComputePointPairPenetration()
            for p in pairs:
                print(p.id_A, p.id_B)
            
            return query_object.HasCollisions()
        
        def isStateValid(state):
            """ wrapper around is_colliding that is passed into ompl"""
            # parse state into q
            q = self.parse_state(state)
            # the state is valid if there are no collisions
            return not is_colliding(q)

        joint_limits = self.joint_limits() 

        space = ob.RealVectorStateSpace(self.nq)
        bounds = ob.RealVectorBounds(self.nq)

        for i in range(self.nq):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])

        space.setBounds(bounds)
        ss = og.SimpleSetup(space)
        ss.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))
        start = self.q_to_state(space, q_start)
        assert isStateValid(start), "you have supplied an invalid start state RRT"
        goal = self.q_to_state(space, q_goal)
        assert isStateValid(goal), "you have supplied an invalid goal state for RRT"
        ss.setStartAndGoalStates(start, goal) 
        solved = ss.solve()

        assert solved

        ss.simplifySolution()
        path = ss.getSolutionPath()

        res = []
        costs = []
        for state in path.getStates():
            q = self.parse_state(state)
            if len(costs) == 0:
                costs.append(0)
            else:
                d = q - res[-1]
                cost = np.sqrt(d.dot(d))
                costs.append(costs[-1] + cost)
            res.append(q)
        
        
        if (costs[-1] == 0):
            costs[-1] = 1

        qs = np.array(res)
        times = start_time + ((goal_time - start_time)*np.array(costs)/costs[-1])

        if len(qs) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                    qs[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                qs[:, 0:self.nq].T)
        

    def joint_limits(self):
        joint_inds = self.plant.GetJointIndices(self.panda)[:self.nq]
        joint_limits = []
        for i in joint_inds:
            joint = self.plant.get_joint(i)
            joint_limits.append((joint.position_lower_limits()[0],
                                 joint.position_upper_limits()[0]))
        return joint_limits




    def parse_state(self, state):
        """ parses ompl RealVectorStateSpace::StateType into a numpy array"""
        q = []
        for i in range(self.nq):
            q.append(state[i])
        return np.array(q)

    def q_to_state(self, space, q):
        """ turns a python list q into an ompl state"""
        state = ob.State(space)
        for i in range(len(q)):
            state[i] = q[i]
        return state
            
        
    def pregrasp_q(self):
        ''' find a good pregrasp pose (self.q_P) given the grasping pose '''
        assert self.q_G is not None
        # find the current grasping pose
        X_Grasp = self.q_to_X_WG(self.q_G)
        
        # retreat in the negative z direction of the gripper frame
        X_Pregrasp = X_Grasp.multiply(RigidTransform(RotationMatrix(), [0, 0, 0]))
        q = self.X_WG_to_q(X_Pregrasp)

        return q, X_Pregrasp, X_Grasp
        
    def q_to_X_WG(self, q):
        assert len(q) == self.nq
        
        self.plant.SetPositions(self.plant_context, self.panda, q)
        #self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName("hand"), [-0.04, 0.04])
        
        body = self.plant.GetBodyByName("panda_hand")
        return self.plant.EvalBodyPoseInWorld(self.plant_context, body)
        
        
    def X_WG_to_q(self, X_WG, warm_start = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])):
        ik_station, _ = create_welded_station(self.station, self.station_context)
        plant = ik_station.get_multibody_plant()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station.CreateDefaultContext())
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
                plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                plant.world_frame(),
                X_WG.translation() - self.p_tol,
                X_WG.translation() + self.p_tol)
        ik.AddOrientationConstraint(
                plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                plant.world_frame(),
                X_WG.rotation(),
                self.theta_tol)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        q = ik.q()
        prog = ik.prog()
        prog.AddQuadraticErrorCost(np.identity(len(q)), self.q_nominal, q)
        prog.SetInitialGuess(q, warm_start)
        result = Solve(prog)
        if not result.is_success():
            print("failed pregrasp ik, returning best result")
        return result.GetSolution(q)

def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.15]

def random_orientation():
    rot = RotationMatrix.MakeZRotation(np.random.uniform(-np.pi, np.pi))
    rot = rot.multiply(RotationMatrix.MakeXRotation(-np.pi/2))
    return rot

In [7]:
builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(PandaStation(time_step = 0.001))
station.SetupTableStation()
item = "soup"
station.AddModelFromFile(ycb_resource(item), 
                        RigidTransform(RotationMatrix.MakeZRotation(np.pi/2), [0.6, 0, 0.05]),
                        name = "soup")

plant = station.get_multibody_plant()
panda = station.GetPanda()
station.Finalize()

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)

start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))


meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url, role = Role.kProximity)# <- this commented part allows visualization of the collisions

grasp_finder = builder.AddSystem(FindGrasp(station, station_context))
picker = builder.AddSystem(Picker(station, station_context))

builder.Connect(grasp_finder.GetOutputPort("panda_grasp_position"), picker.GetInputPort("grasp_position"))
builder.Connect(station.GetOutputPort("panda_position_measured"), picker.GetInputPort("panda_position"))
builder.Connect(picker.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
builder.Connect(picker.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

diagram = builder.Build()

#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
#station.GetInputPort("hand_position").FixValue(station_context, [0.08]) # taking the desired hand separation
#station.GetInputPort("panda_position").FixValue(station_context, [0.0, 0.1, 0.0, -1.2, 0, 1.6, 0])


meshcat.start_recording()

simulator.AdvanceTo(10.5)

meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6011...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/
Connected to meshcat-server.
INITIALIZATION
q: [ 6.70045331e-03  5.31858887e-01 -6.62235458e-03 -2.02546525e+00
  6.08938807e-03  2.55730572e+00  4.46924340e-04]
cost -0.7071067811863091
panda ids:
<GeometryId value=1668>
<GeometryId value=1670>
<GeometryId value=1672>
<GeometryId value=1674>
<GeometryId value=1676>
<GeometryId value=1678>
<GeometryId value=1680>
<GeometryId value=1682>
<GeometryId value=1684>
<GeometryId value=1686>
<GeometryId value=1688>
<GeometryId value=1690>
<GeometryId value=1692>
<GeometryId value=1694>
<GeometryId value=1698>
<GeometryId value=1700>
<GeometryId value=1702>
<GeometryId value=1704>
<GeometryId value=1706>
<GeometryId value=1708>
<GeometryId value=1712>
<GeometryId value=1714>
<GeometryId value=1716>
<GeometryId value=1718>
<GeometryId value=1720>
<GeometryId value=1722>
<GeometryId value=1726>
<

In [ ]:
#file = open(item + ".html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

#subprocess.check_output(["tar", "vczf", "media/videos/ycb_pick/" + item + ".gz", item + ".html"])
#subprocess.check_output(["trash", item +".html"])